In [1]:
!nvidia-smi

Wed Mar  9 05:43:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    36W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import numpy as np


In [3]:
# Get the input data
input = np.array([
                  [32,55,77],
                  [31, 75,57],
                  [52,55,77],
                  [22,80,87],
                  [62,80,77]
], dtype = 'float32')

In [4]:
input

array([[32., 55., 77.],
       [31., 75., 57.],
       [52., 55., 77.],
       [22., 80., 87.],
       [62., 80., 77.]], dtype=float32)

In [5]:
targets = np.array([
                    [32, 66],
                    [52, 76],
                    [102, 155],
                    [32, 26],
                    [99, 100],
], dtype= 'float32')

In [6]:
targets

array([[ 32.,  66.],
       [ 52.,  76.],
       [102., 155.],
       [ 32.,  26.],
       [ 99., 100.]], dtype=float32)

Convert the numpy data into tensors for pytorch

In [7]:
input_tensors = torch.from_numpy(input)
input_tensors

tensor([[32., 55., 77.],
        [31., 75., 57.],
        [52., 55., 77.],
        [22., 80., 87.],
        [62., 80., 77.]])

In [8]:
target_tensors = torch.from_numpy(targets)
target_tensors

tensor([[ 32.,  66.],
        [ 52.,  76.],
        [102., 155.],
        [ 32.,  26.],
        [ 99., 100.]])

Initialize weights and biases

In [9]:
w = torch.randn(2,3, requires_grad = True)
b = torch.randn(2, requires_grad = True)
w, b

(tensor([[-1.2550,  0.2177,  1.1017],
         [-0.6025, -0.3893, -1.0774]], requires_grad=True),
 tensor([-1.5629, -0.4115], requires_grad=True))

To be able to multiply x(inputs) and w we have to take transpose of w because w is of shape 2x3 and x is of shape nx3. Therefore transpose of w will make the shape 3X2 which is required for matrix multiplication.

In [10]:
def model(x):
  return x @ w.t() + b  #@ for dot product

In [11]:
prediction = model(input_tensors
                   )
prediction

tensor([[  55.0839, -124.0591],
        [  38.6580, -109.6944],
        [  29.9837, -136.1083],
        [  84.0937, -138.5403],
        [  22.8760, -151.8648]], grad_fn=<AddBackward0>)

In [12]:
# Find the loss using loss functions

def MSE(t1, t2):
  diff = t1- t2
  return torch.sum(diff *diff)/ diff.numel()    # numel() returns number of elements

In [13]:
loss = MSE(prediction, target_tensors)
loss

tensor(26026.4121, grad_fn=<DivBackward0>)

In [14]:
# Compute gradient

loss.backward()

In [15]:
print(w)
print(w.grad)

tensor([[-1.2550,  0.2177,  1.1017],
        [-0.6025, -0.3893, -1.0774]], requires_grad=True)
tensor([[ -1398.6787,  -1122.8721,  -1171.5378],
        [ -9242.3105, -14740.7383, -16268.6113]])


In [16]:
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5

  # Using torch.no grad because we do not wish to calculate the gradient here, only update weights and biases



In [17]:
print(w)

tensor([[-1.2410,  0.2289,  1.1135],
        [-0.5100, -0.2419, -0.9147]], requires_grad=True)


In [18]:
preds = model(input_tensors)
preds

tensor([[  57.0513, -100.4652],
        [  40.6017,  -86.4985],
        [  32.2309, -110.6659],
        [  86.3191, -110.5585],
        [  25.5437, -121.8130]], grad_fn=<AddBackward0>)

In [19]:
loss = MSE(preds, target_tensors)
loss

tensor(20651.5605, grad_fn=<DivBackward0>)

Training stage

In [20]:
n = 1000
for i in range(n):
  preds = model(input_tensors)
  loss = MSE(preds, target_tensors)
  loss.backward()
  with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()  #set grad = 0 after every iteration
    b.grad.zero_()
  print(f'Epoch is {i}/{n}.........................loss = {loss}')

Epoch is 0/1000.........................loss = 20651.560546875
Epoch is 1/1000.........................loss = 12418.744140625
Epoch is 2/1000.........................loss = 10129.75
Epoch is 3/1000.........................loss = 8358.9169921875
Epoch is 4/1000.........................loss = 6988.5283203125
Epoch is 5/1000.........................loss = 5927.61279296875
Epoch is 6/1000.........................loss = 5105.869140625
Epoch is 7/1000.........................loss = 4468.96728515625
Epoch is 8/1000.........................loss = 3974.91943359375
Epoch is 9/1000.........................loss = 3591.278076171875
Epoch is 10/1000.........................loss = 3292.966796875
Epoch is 11/1000.........................loss = 3060.607421875
Epoch is 12/1000.........................loss = 2879.22412109375
Epoch is 13/1000.........................loss = 2737.24267578125
Epoch is 14/1000.........................loss = 2625.71875
Epoch is 15/1000.........................loss = 2537.73999

In [21]:
preds = model(input_tensors)
preds

tensor([[ 53.4892,  72.0928],
        [ 46.2134,  54.7652],
        [ 84.7741, 117.2271],
        [ 36.1975,  41.7443],
        [ 97.1988, 128.8537]], grad_fn=<AddBackward0>)

In [22]:
target_tensors

tensor([[ 32.,  66.],
        [ 52.,  76.],
        [102., 155.],
        [ 32.,  26.],
        [ 99., 100.]])

In [23]:
loss = MSE(target_tensors, preds)
loss

tensor(380.8109, grad_fn=<DivBackward0>)

LR using pytorch inbuilt function

In [24]:
inputs = np.array([
                  [73,67,43],
                  [91,88,64],
                  [87,134,58],
                  [102,43,37],
                  [69,96, 70],
                  [74,66,43],
                  [91,87,65],
                  [88,134,59],
                  [101,44,37],
                  [68,96,71],
                  [73,66,44],
                  [92,87,64],
                  [87, 135, 57],
                  [103, 43, 36],
                  [68,97,70]], 
                  dtype = 'float32')

targets = np.array([
                  [56, 70],
                  [81,101],
                  [119,133],
                  [22,37],
                  [103,119],
                  [57,69],
                  [80,102],
                  [118,132],
                  [21,38],
                  [104,118],
                  [57,69],
                  [82,100],
                  [118,134],
                  [20,38],
                  [102,120]],
                   dtype = 'float32') 

input_tensors = torch.from_numpy(inputs)
target_tensors = torch.from_numpy(targets)

In [25]:
#Data loader

from torch.utils.data import TensorDataset

In [26]:
train_ds = TensorDataset(input_tensors, target_tensors)
train_ds[0: 3]    #if you want to select some amount of inputs and targets

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [27]:
from torch.utils.data import DataLoader

batch_size = 7
train_data = DataLoader(train_ds, batch_size, shuffle= True)

In [28]:
for xb, yb in train_data:
  print(xb)
  print(yb)
  break

tensor([[ 88., 134.,  59.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.],
        [101.,  44.,  37.],
        [ 87., 135.,  57.],
        [ 92.,  87.,  64.],
        [102.,  43.,  37.]])
tensor([[118., 132.],
        [ 20.,  38.],
        [102., 120.],
        [ 21.,  38.],
        [118., 134.],
        [ 82., 100.],
        [ 22.,  37.]])


Build a linear model using pytorch inbuilt module called nn

In [29]:
#define model here
model = torch.nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4757,  0.0644, -0.0616],
        [ 0.2138,  0.1934,  0.2381]], requires_grad=True)
Parameter containing:
tensor([ 0.5297, -0.2530], requires_grad=True)


In [30]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.4757,  0.0644, -0.0616],
         [ 0.2138,  0.1934,  0.2381]], requires_grad=True),
 Parameter containing:
 tensor([ 0.5297, -0.2530], requires_grad=True)]

In [31]:
preds = model(input_tensors)
preds

tensor([[-32.5292,  38.5482],
        [-41.0322,  51.4579],
        [-35.7954,  58.0705],
        [-47.5031,  38.6766],
        [-30.4197,  49.7313],
        [-33.0694,  38.5686],
        [-41.1582,  51.5026],
        [-36.3327,  58.5224],
        [-46.9629,  38.6562],
        [-30.0055,  49.7557],
        [-32.6553,  38.5930],
        [-41.5724,  51.4782],
        [-35.6694,  58.0258],
        [-47.9173,  38.6522],
        [-29.8795,  49.7110]], grad_fn=<AddmmBackward0>)

In [32]:
import torch.nn.functional as F
loss_func = F.mse_loss

In [33]:
loss = loss_func(preds, target_tensors)
loss

tensor(8294.0146, grad_fn=<MseLossBackward0>)

In [34]:
optimizer = torch.optim.SGD(model.parameters(), lr= 1e-5)

In [46]:
#training loop
def fit(epochs, model, loss_func, optimizer, train_data):
  for idx, epoch in enumerate(range(epochs+1)):
    for xb, yb in train_data:
      pred = model(xb)
      loss = loss_func(pred, yb)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    if idx % 10 == 0:
      print(f'Epoch{idx}/{epochs}\t\tloss: {loss}')

In [47]:
num_epochs = 1000
fit(num_epochs, model, loss_func, optimizer, train_data)

Epoch0/1000		loss: 1.3247087001800537
Epoch10/1000		loss: 0.36041024327278137
Epoch20/1000		loss: 0.9511488080024719
Epoch30/1000		loss: 0.8266086578369141
Epoch40/1000		loss: 1.2929682731628418
Epoch50/1000		loss: 1.2714500427246094
Epoch60/1000		loss: 0.6894022822380066
Epoch70/1000		loss: 0.9731493592262268
Epoch80/1000		loss: 0.38048532605171204
Epoch90/1000		loss: 0.7431103587150574
Epoch100/1000		loss: 3.0769457817077637
Epoch110/1000		loss: 0.8401251435279846
Epoch120/1000		loss: 1.047459363937378
Epoch130/1000		loss: 0.9311254620552063
Epoch140/1000		loss: 0.4516870081424713
Epoch150/1000		loss: 1.1204757690429688
Epoch160/1000		loss: 0.6363790035247803
Epoch170/1000		loss: 0.5118137001991272
Epoch180/1000		loss: 1.040936827659607
Epoch190/1000		loss: 0.8655748963356018
Epoch200/1000		loss: 0.7629744410514832
Epoch210/1000		loss: 1.163097620010376
Epoch220/1000		loss: 0.7536499500274658
Epoch230/1000		loss: 0.40516188740730286
Epoch240/1000		loss: 0.973793625831604
Epoch250/100

In [49]:
# for a custom prediction try

model(torch.tensor([[57,77,65.]]))

tensor([[ 87.2535, 102.5167]], grad_fn=<AddmmBackward0>)

In [50]:
from sklearn.metrics import r2_score

To save the model to your google drive. First mount your drive

In [51]:
torch.save(model.state_dict(), 'LR_w_pyt.pth')

In [52]:
#If you want to load the model
torch.load('LR_w_pyt.pth')

OrderedDict([('weight', tensor([[-0.4091,  0.8421,  0.6952],
                      [-0.2858,  0.7948,  0.8902]])),
             ('bias', tensor([ 0.5364, -0.2564]))])